In [1]:
import numpy as np 
import pandas as pd
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.classify import SklearnClassifier
import matplotlib.pyplot as plt
import string
import re
from collections import Counter
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score


[nltk_data] Downloading package stopwords to /Users/Manam/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/Manam/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
df = pd.read_csv('vaderDataset.csv')
df

,Unnamed: 0,Unnamed: 0.1,Tweets,Date,Country,Vader-sentiment value,Vader-sentiment
0,0,0,I wish quick recovery for talented @TOLOnews J...,2020-04-29 23:33:40+00:00,afghanistan,0.9022,positive
1,1,1,These chinese people on campus are so consider...,2020-01-28 23:41:13,afghanistan,0.5777,positive
2,2,2,"China’s Leader, Under Fire, Says He Led Corona...",2020-02-15 23:17:09,afghanistan,-0.6124,negative
3,3,3,RT @DarrenEuronews: Top 5 countries 🌍 with hig...,Sat Jun 13 07:50:54 +0000 2020,afghanistan,0.2732,positive
4,4,4,"RT @MileyCyrus: Spain, you united in solidarit...",Sat Jun 13 09:52:29 +0000 2020,afghanistan,0.6249,positive
5,5,5,RT @_junaidahmad_: Inshallah Get well soon !! ...,Sat Jun 13 10:00:38 +0000 2020,afghanistan,0.7835,positive
6,6,6,"RT @UNICEF: Singing 🎤, filming 🎥 and learning ...",Sat Jun 13 12:51:28 +0000 2020,afghanistan,0.4019,positive
7,7,7,I am proud of my spritual brothers and sisters...,Sat Jun 13 14:16:52 +0000 2020,afghanistan,0.7096,positive
8,8,8,Just finished co-leading an @IBWellsSociety vi...,2020-04-29 23:21:09+00:00,albania,0.9441,positive
9,9,9,‘It’s going to disappear’: Trump’s changing to...,2020-04-29 23:04:41+00:00,albania,0.0000,Neutral


In [3]:
df=df.drop(['Unnamed: 0','Unnamed: 0.1'],axis=1)

In [4]:
stopword = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()
wn = nltk.WordNetLemmatizer()

In [6]:
df['Tweets']=df['Tweets'].astype('str')

In [7]:
def clean_text(text):
    text_lc = "".join([word.lower() for word in text if word not in string.punctuation]) # remove puntuation
    text_rc = re.sub('[0-9]+', '', text_lc)
    tokens = re.split('\W+', text_rc)    # tokenization
    text = [word for word in tokens if word not in stopword]  # remove stopwords and stemming
    return text
df['Text_cleaned'] = df['Tweets'].apply(lambda x: clean_text(x))

In [8]:
def stemming(text):
    text = [ps.stem(word) for word in text]
    return text

df['Text_stemmed'] = df['Text_cleaned'].apply(lambda x: stemming(x))

def lemmatizer(text):
    text = [wn.lemmatize(word) for word in text]
    return text
df['Text_lemmatized_stemmed'] = df['Text_stemmed'].apply(lambda x: lemmatizer(x))
df.head()


,Tweets,Date,Country,Vader-sentiment value,Vader-sentiment,Text_cleaned,Text_stemmed,Text_lemmatized_stemmed
0,I wish quick recovery for talented @TOLOnews J...,2020-04-29 23:33:40+00:00,afghanistan,0.9022,positive,"[wish, quick, recovery, talented, tolonews, jo...","[wish, quick, recoveri, talent, tolonew, journ...","[wish, quick, recoveri, talent, tolonew, journ..."
1,These chinese people on campus are so consider...,2020-01-28 23:41:13,afghanistan,0.5777,positive,"[chinese, people, campus, considerate, wear, m...","[chines, peopl, campu, consider, wear, mask, o...","[chine, peopl, campu, consider, wear, mask, ot..."
2,"China’s Leader, Under Fire, Says He Led Corona...",2020-02-15 23:17:09,afghanistan,-0.6124,negative,"[china, leader, fire, says, led, coronavirus, ...","[china, leader, fire, say, led, coronaviru, fi...","[china, leader, fire, say, led, coronaviru, fi..."
3,RT @DarrenEuronews: Top 5 countries 🌍 with hig...,Sat Jun 13 07:50:54 +0000 2020,afghanistan,0.2732,positive,"[rt, darreneuronews, top, countries, highest, ...","[rt, darreneuronew, top, countri, highest, num...","[rt, darreneuronew, top, countri, highest, num..."
4,"RT @MileyCyrus: Spain, you united in solidarit...",Sat Jun 13 09:52:29 +0000 2020,afghanistan,0.6249,positive,"[rt, mileycyrus, spain, united, solidarity, bl...","[rt, mileycyru, spain, unit, solidar, black, l...","[rt, mileycyru, spain, unit, solidar, black, l..."


In [9]:
from sklearn.preprocessing import LabelEncoder 
from sklearn.preprocessing import OneHotEncoder
le = LabelEncoder() 
#df['deceptive']= le.fit_transform(df['deceptive'])
df['polarity']= le.fit_transform(df['Vader-sentiment'])



In [10]:
y = df['polarity']
X = df['Text_lemmatized_stemmed']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train

23049     [china, quarantin, wuhan, shut, airport, publi...
53095      [fauci, gilead, remdesivir, block, coronaviru, ]
49356     [rt, kattanew, maharashtra, state, govern, red...
88116     [coronaviru, latest, public, health, england, ...
96239     [case, confus, corona, beer, noth, coronaviru,...
93028                [asian, peopl, corona, viru, eat, dog]
24308     [coronaviru, epidem, updat, antivir, drug, tre...
4115      [someon, left, unopen, bag, crisp, train, coro...
10399     [malaysia, make, posit, progress, contain, cor...
101686    [coronaviru, death, toll, surpass, new, death,...
6235      [second, day, coronaviru, home, quarantin, lik...
65968     [midnight, pub, across, road, live, glossop, c...
53117     [coronaviru, brief, zoom, hydroxychloroquin, t...
37493     [scottmorrisonmp, gladysb, enforc, endgam, c, ...
29776     [someon, live, thru, wischolera, honest, god, ...
107144    [corona, viru, gonna, turn, hollieclapp, even,...
112502    [wish, american, believ, coron

In [12]:
from tensorflow.python.keras.preprocessing import sequence
from tensorflow.python.keras.preprocessing import text

TOP_K = 30000

MAX_SEQUENCE_LENGTH = 10000

def sequence_vectorize(train_texts, val_texts):
    tokenizer = text.Tokenizer(num_words=TOP_K)
    tokenizer.fit_on_texts(train_texts)

    x_train = tokenizer.texts_to_sequences(train_texts)
    x_val = tokenizer.texts_to_sequences(val_texts)

    max_length = len(max(x_train, key=len))
    if max_length > MAX_SEQUENCE_LENGTH:
        max_length = MAX_SEQUENCE_LENGTH

    x_train = sequence.pad_sequences(x_train, maxlen=max_length)
    x_val = sequence.pad_sequences(x_val, maxlen=max_length)
    return x_train, x_val, tokenizer.word_index, max_length

In [14]:
from keras.models import Model, Sequential
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping

In [15]:
x_train, x_val = X_train.values, X_test.values
x_train, x_val, word_index, max_length = sequence_vectorize(x_train, x_val)
num_features = min(len(word_index) + 1, TOP_K)

In [31]:

import tensorflow as tf
embedding_vector_length = 256

model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(TOP_K, embedding_vector_length, input_length=max_length),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x_train, y_train, validation_data=(x_val, y_test), epochs=1000, batch_size=128)

Epoch 1/1000
710/710 [==============================] - 119s 168ms/step - loss: -2.6585 - accuracy: 0.4848 - val_loss: -5.7857 - val_accuracy: 0.5621
Epoch 2/1000
710/710 [==============================] - 3585s 5s/step - loss: -8.3552 - accuracy: 0.5496 - val_loss: -10.6490 - val_accuracy: 0.5595
Epoch 3/1000
710/710 [==============================] - 157s 222ms/step - loss: -13.5058 - accuracy: 0.5484 - val_loss: -15.3163 - val_accuracy: 0.5572
Epoch 4/1000
710/710 [==============================] - 135s 190ms/step - loss: -18.6956 - accuracy: 0.5519 - val_loss: -19.7504 - val_accuracy: 0.5493
Epoch 5/1000
710/710 [==============================] - 113s 159ms/step - loss: -23.9239 - accuracy: 0.5543 - val_loss: -24.3115 - val_accuracy: 0.5477
Epoch 6/1000
710/710 [==============================] - 114s 161ms/step - loss: -29.2936 - accuracy: 0.5546 - val_loss: -28.4925 - val_accuracy: 0.5444
Epoch 7/1000
710/710 [==============================] - 114s 160ms/step - loss: -35.0738 - ac

710/710 [==============================] - 112s 158ms/step - loss: -320.7173 - accuracy: 0.5732 - val_loss: -230.6051 - val_accuracy: 0.5623
Epoch 55/1000
710/710 [==============================] - 112s 158ms/step - loss: -327.3591 - accuracy: 0.5743 - val_loss: -235.7710 - val_accuracy: 0.5650
Epoch 56/1000
710/710 [==============================] - 114s 160ms/step - loss: -333.4516 - accuracy: 0.5761 - val_loss: -239.4234 - val_accuracy: 0.5648
Epoch 57/1000
710/710 [==============================] - 106s 150ms/step - loss: -338.7773 - accuracy: 0.5744 - val_loss: -242.8559 - val_accuracy: 0.5631
Epoch 58/1000
710/710 [==============================] - 107s 151ms/step - loss: -344.8363 - accuracy: 0.5706 - val_loss: -248.2832 - val_accuracy: 0.5655
Epoch 59/1000
710/710 [==============================] - 110s 155ms/step - loss: -352.0590 - accuracy: 0.5743 - val_loss: -249.5157 - val_accuracy: 0.5619
Epoch 60/1000
710/710 [==============================] - 117s 165ms/step - loss: -35

Epoch 107/1000
710/710 [==============================] - 124s 175ms/step - loss: -651.5580 - accuracy: 0.5830 - val_loss: -451.5698 - val_accuracy: 0.5622
Epoch 108/1000
710/710 [==============================] - 118s 167ms/step - loss: -660.9999 - accuracy: 0.5834 - val_loss: -452.4270 - val_accuracy: 0.5636
Epoch 109/1000
710/710 [==============================] - 123s 173ms/step - loss: -666.5715 - accuracy: 0.5819 - val_loss: -455.3858 - val_accuracy: 0.5666
Epoch 110/1000
710/710 [==============================] - 119s 167ms/step - loss: -671.6085 - accuracy: 0.5807 - val_loss: -462.5189 - val_accuracy: 0.5634
Epoch 111/1000
710/710 [==============================] - 122s 172ms/step - loss: -677.7142 - accuracy: 0.5813 - val_loss: -465.0641 - val_accuracy: 0.5623
Epoch 112/1000
710/710 [==============================] - 121s 171ms/step - loss: -685.7951 - accuracy: 0.5839 - val_loss: -468.9673 - val_accuracy: 0.5605
Epoch 113/1000
710/710 [==============================] - 117s 1

710/710 [==============================] - 140s 197ms/step - loss: -980.6348 - accuracy: 0.5831 - val_loss: -657.1827 - val_accuracy: 0.5566
Epoch 160/1000
710/710 [==============================] - 172s 242ms/step - loss: -987.6516 - accuracy: 0.5819 - val_loss: -663.7927 - val_accuracy: 0.5584
Epoch 161/1000
710/710 [==============================] - 189s 266ms/step - loss: -993.5642 - accuracy: 0.5855 - val_loss: -669.3317 - val_accuracy: 0.5565
Epoch 162/1000
710/710 [==============================] - 190s 268ms/step - loss: -998.9057 - accuracy: 0.5869 - val_loss: -667.4125 - val_accuracy: 0.5573
Epoch 163/1000
710/710 [==============================] - 188s 265ms/step - loss: -1006.4863 - accuracy: 0.5853 - val_loss: -672.0831 - val_accuracy: 0.5585
Epoch 164/1000
710/710 [==============================] - 191s 269ms/step - loss: -1013.5870 - accuracy: 0.5874 - val_loss: -680.9731 - val_accuracy: 0.5583
Epoch 165/1000
710/710 [==============================] - 191s 269ms/step - l

710/710 [==============================] - 114s 161ms/step - loss: -1312.7096 - accuracy: 0.5954 - val_loss: -857.7850 - val_accuracy: 0.5605
Epoch 212/1000
710/710 [==============================] - 118s 166ms/step - loss: -1322.2461 - accuracy: 0.5952 - val_loss: -869.5259 - val_accuracy: 0.5621
Epoch 213/1000
710/710 [==============================] - 123s 173ms/step - loss: -1325.1942 - accuracy: 0.5975 - val_loss: -875.5401 - val_accuracy: 0.5587
Epoch 214/1000
710/710 [==============================] - 114s 161ms/step - loss: -1328.1257 - accuracy: 0.5899 - val_loss: -878.2240 - val_accuracy: 0.5601
Epoch 215/1000
710/710 [==============================] - 123s 174ms/step - loss: -1336.8721 - accuracy: 0.5946 - val_loss: -879.7712 - val_accuracy: 0.5594
Epoch 216/1000
710/710 [==============================] - 116s 163ms/step - loss: -1343.2340 - accuracy: 0.5962 - val_loss: -891.8561 - val_accuracy: 0.5599
Epoch 217/1000
710/710 [==============================] - 121s 170ms/step

...

Epoch 783/1000
710/710 [==============================] - 129s 182ms/step - loss: -4938.8457 - accuracy: 0.6074 - val_loss: -3142.9766 - val_accuracy: 0.5553
Epoch 784/1000
710/710 [==============================] - 118s 166ms/step - loss: -4966.0786 - accuracy: 0.6109 - val_loss: -3150.6533 - val_accuracy: 0.5552
Epoch 785/1000
710/710 [==============================] - 108s 152ms/step - loss: -4961.4629 - accuracy: 0.6126 - val_loss: -3139.8538 - val_accuracy: 0.5541
Epoch 786/1000
710/710 [==============================] - 105s 147ms/step - loss: -4957.4702 - accuracy: 0.6092 - val_loss: -3158.4719 - val_accuracy: 0.5557
Epoch 787/1000
710/710 [==============================] - 112s 158ms/step - loss: -4978.9722 - accuracy: 0.6076 - val_loss: -3155.2212 - val_accuracy: 0.5543
Epoch 788/1000
710/710 [==============================] - 123s 173ms/step - loss: -4977.7275 - accuracy: 0.6080 - val_loss: -3164.2773 - val_accuracy: 0.5544
Epoch 789/1000
710/710 [============================

710/710 [==============================] - 153s 215ms/step - loss: -5269.5186 - accuracy: 0.6066 - val_loss: -3326.1831 - val_accuracy: 0.5541
Epoch 835/1000
710/710 [==============================] - 179s 252ms/step - loss: -5269.3721 - accuracy: 0.6102 - val_loss: -3339.8518 - val_accuracy: 0.5536
Epoch 836/1000
710/710 [==============================] - 139s 196ms/step - loss: -5279.5483 - accuracy: 0.6098 - val_loss: -3340.9468 - val_accuracy: 0.5558
Epoch 837/1000
710/710 [==============================] - 171s 241ms/step - loss: -5302.7583 - accuracy: 0.6104 - val_loss: -3352.1013 - val_accuracy: 0.5559
Epoch 838/1000
710/710 [==============================] - 156s 219ms/step - loss: -5304.8228 - accuracy: 0.6130 - val_loss: -3336.5635 - val_accuracy: 0.5543
Epoch 839/1000
710/710 [==============================] - 149s 209ms/step - loss: -5291.4116 - accuracy: 0.6100 - val_loss: -3355.0837 - val_accuracy: 0.5553
Epoch 840/1000
710/710 [==============================] - 145s 205m

Epoch 886/1000
710/710 [==============================] - 127s 179ms/step - loss: -5591.0146 - accuracy: 0.6111 - val_loss: -3562.9841 - val_accuracy: 0.5552
Epoch 887/1000
710/710 [==============================] - 120s 169ms/step - loss: -5613.3447 - accuracy: 0.6117 - val_loss: -3574.8682 - val_accuracy: 0.5566
Epoch 888/1000
710/710 [==============================] - 127s 178ms/step - loss: -5621.6108 - accuracy: 0.6087 - val_loss: -3565.1326 - val_accuracy: 0.5530
Epoch 889/1000
710/710 [==============================] - 126s 177ms/step - loss: -5622.4956 - accuracy: 0.6115 - val_loss: -3564.6428 - val_accuracy: 0.5535
Epoch 890/1000
710/710 [==============================] - 1125s 2s/step - loss: -5626.5566 - accuracy: 0.6105 - val_loss: -3586.5974 - val_accuracy: 0.5537
Epoch 891/1000
710/710 [==============================] - 125s 176ms/step - loss: -5639.4692 - accuracy: 0.6099 - val_loss: -3572.0540 - val_accuracy: 0.5547
Epoch 892/1000
710/710 [==============================

Epoch 938/1000
710/710 [==============================] - 120s 169ms/step - loss: -5940.4512 - accuracy: 0.6123 - val_loss: -3776.2944 - val_accuracy: 0.5522
Epoch 939/1000
710/710 [==============================] - 124s 174ms/step - loss: -5961.3086 - accuracy: 0.6109 - val_loss: -3759.4324 - val_accuracy: 0.5518
Epoch 940/1000
710/710 [==============================] - 122s 171ms/step - loss: -5948.2231 - accuracy: 0.6103 - val_loss: -3783.8352 - val_accuracy: 0.5524
Epoch 941/1000
710/710 [==============================] - 115s 162ms/step - loss: -5980.4443 - accuracy: 0.6106 - val_loss: -3767.5928 - val_accuracy: 0.5514
Epoch 942/1000
710/710 [==============================] - 120s 169ms/step - loss: -5950.5322 - accuracy: 0.6127 - val_loss: -3778.0833 - val_accuracy: 0.5521
Epoch 943/1000
710/710 [==============================] - 120s 169ms/step - loss: -5958.3145 - accuracy: 0.6116 - val_loss: -3789.2026 - val_accuracy: 0.5536
Epoch 944/1000
710/710 [============================

Epoch 990/1000
710/710 [==============================] - 120s 169ms/step - loss: -6265.6450 - accuracy: 0.6089 - val_loss: -3963.8088 - val_accuracy: 0.5525
Epoch 991/1000
710/710 [==============================] - 124s 174ms/step - loss: -6288.9341 - accuracy: 0.6109 - val_loss: -3951.2019 - val_accuracy: 0.5516
Epoch 992/1000
710/710 [==============================] - 125s 176ms/step - loss: -6276.9888 - accuracy: 0.6107 - val_loss: -3952.3706 - val_accuracy: 0.5501
Epoch 993/1000
710/710 [==============================] - 122s 172ms/step - loss: -6291.3423 - accuracy: 0.6101 - val_loss: -3958.1724 - val_accuracy: 0.5515
Epoch 994/1000
710/710 [==============================] - 124s 175ms/step - loss: -6297.0898 - accuracy: 0.6075 - val_loss: -3949.9905 - val_accuracy: 0.5505
Epoch 995/1000
710/710 [==============================] - 129s 181ms/step - loss: -6310.0723 - accuracy: 0.6115 - val_loss: -3982.3994 - val_accuracy: 0.5502
Epoch 996/1000
710/710 [============================

In [32]:
y_pred = model.predict(x_val)
y_pred = [1 if a > 0.5 else 0 for a in y_pred]

print(confusion_matrix(list(y_test), list(y_pred)))

f1_score(list(y_test), y_pred, average='weighted')

[[6413 2392    0]
 [1191 6145    0]
 [ 455 6119    0]]


0.4752953629774532

In [33]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

[[6413 2392    0]
 [1191 6145    0]
 [ 455 6119    0]]
              precision    recall  f1-score   support

           0       0.80      0.73      0.76      8805
           1       0.42      0.84      0.56      7336
           2       0.00      0.00      0.00      6574

    accuracy                           0.55     22715
   macro avg       0.41      0.52      0.44     22715
weighted avg       0.44      0.55      0.48     22715

0.5528505392912173
